In [1]:
!pip install cellpose

  Using cached cellpose-3.0.7-py3-none-any.whl.metadata (23 kB)
  Using cached natsort-8.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached torch-2.2.2-cp311-none-macosx_11_0_arm64.whl.metadata (25 kB)
  Using cached opencv_python_headless-4.9.0.80-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
  Using cached fastremap-1.14.1-cp311-cp311-macosx_10_9_universal2.whl.metadata (9.7 kB)
  Using cached roifile-2024.3.20-py3-none-any.whl.metadata (4.6 kB)
Using cached cellpose-3.0.7-py3-none-any.whl (204 kB)
Using cached torch-2.2.2-cp311-none-macosx_11_0_arm64.whl (59.7 MB)
Using cached fastremap-1.14.1-cp311-cp311-macosx_10_9_universal2.whl (1.5 MB)
Using cached natsort-8.4.0-py3-none-any.whl (38 kB)
Using cached opencv_python_headless-4.9.0.80-cp37-abi3-macosx_11_0_arm64.whl (35.4 MB)
Using cached roifile-2024.3.20-py3-none-any.whl (16 kB)


In [3]:
!pip freeze > requirements_pip.txt

In [1]:
!which conda

/opt/anaconda3/condabin/conda


In [2]:
!pip install "opencv-python-headless<4.3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 4.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models, io
from cellpose.io import imread

In [ ]:
#probably will want something like this for .py version
import os
import sys

dirpath = sys.path[0]
os.chdir(dirpath)

In [4]:
#this is for colab version
import os
import sys

# YOU MIGHT HAVE TO CHANGE THIS PATH, DEPENDING ON WHERE YOU DOWNLAOD THIS
dirpath = '/content/drive/MyDrive/Kexin and Santi work/Santiago Arcecarpio - Severity of Psoriasis across 300 Kaggle/Segmentation'
os.chdir(dirpath)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Kexin and Santi work/Santiago Arcecarpio - Severity of Psoriasis across 300 Kaggle/Segmentation'

In [40]:
dirpath = 'Desktop/repos/cellular segmentation'

True

In [36]:
#gets lists of file names ignoring hidden files eg '.foo.txt'

files = os.listdir(dirpath+'/Images')
for file in files:
    if file[0] == '.':
        files.remove(file)
files

['IMG_3403.png', 'IMG_3402.png']

In [16]:
io.logger_setup()

# model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
model = models.Cellpose(model_type='cyto3')

# list of files
# PUT PATH TO YOUR FILES HERE!
dirpath = os.getcwd()

files = os.listdir(dirpath+'/Images')
for file in files:
    if file[0] == '.':
        files.remove(file)

imgs = [imread(dirpath+'/Images/'+f) for f in files]

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]. We use grayscale images so set out channels to 0,0
channels = [0,0]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

# if diameter is set to None, the size of the cells is estimated on a per image basis
# you can set the average cell `diameter` in pixels yourself (recommended) -- For our purposes we
# will need all the images to be about as zoomed in as each other, and then manually set the diameter
# diameter can be a list or a single number for all images

masks, flows, styles, diams = model.eval(imgs, diameter=None, channels=channels)

2024-04-17 00:06:51,740 [INFO] WRITING LOG OUTPUT TO /Users/sarce/.cellpose/run.log
2024-04-17 00:06:51,742 [INFO] 
cellpose version: 	3.0.7 
platform:       	darwin 
python version: 	3.11.7 
torch version:  	2.2.2
2024-04-17 00:06:51,743 [INFO] >>>> using CPU
2024-04-17 00:06:51,744 [INFO] >> cyto3 << model set to be used
2024-04-17 00:06:51,745 [INFO] WARNING: MKL version on torch not working/installed - CPU version will be slightly slower.
2024-04-17 00:06:51,745 [INFO] see https://pytorch.org/docs/stable/backends.html?highlight=mkl
2024-04-17 00:06:51,859 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2024-04-17 00:06:52,014 [INFO] channels set to [0, 0]
2024-04-17 00:06:52,015 [INFO] ~~~ FINDING MASKS ~~~
2024-04-17 00:08:38,864 [INFO] 100%|##########| 2/2 [01:46<00:00, 53.42s/it]
2024-04-17 00:08:38,864 [INFO] >>>> TOTAL TIME 106.85 sec


In [41]:
import shutil

if os.path.isdir(dirpath+'/model_predictions'):
    shutil.rmtree(dirpath+'/model_predictions')
    os.mkdir(dirpath+'/model_predictions')
else:
  os.mkdir(dirpath+'/model_predictions')
    
os.chdir(dirpath+'/model_predictions')

io.masks_flows_to_seg(imgs, masks, flows, dirpath+'dragana_email_photos_seg.npy', channels=channels, diams=diams)
io.save_masks(imgs, masks, flows, files, png=True)

In [ ]:
plt.imshow(imgs[1])

In [35]:
os.getcwd()

'/Users/sarce/Desktop/repos/cellular segmentation/model_predictions'

# Now we get the number of ROIS per image and store it

In [42]:
num_rois_list = []
for mask in masks:
    num_rois_list.append(np.max(mask))

import pandas as pd

# Sample transposed list of lists
data = [files, num_rois_list]

# Transpose the list of lists
transposed_data = list(map(list, zip(*data)))

# Convert transposed list of lists to a DataFrame
df = pd.DataFrame(transposed_data)

# Specify the file path to save the CSV file
csv_file_path = 'num_cells.csv'

# Save DataFrame to CSV file
df.to_csv(csv_file_path, index=False)

print("CSV file saved successfully.")

CSV file saved successfully.


/opt/anaconda3/bin/conda


In [44]:
!conda list -e > requirements_checkpoint.txt